In [1]:
import os
import cv2
import numpy as np
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam


Directorio donde se encuentran las huellas

In [2]:
data_directory = '/content/drive/MyDrive/sistema-biometrico'

Tamaño y procesamiento de imagenes

In [3]:
img_width, img_height = 224, 224
# Preprocesamiento de datos
datagen = ImageDataGenerator(rescale=1./255,
                             rotation_range=40,  # Reducir el rango de rotación
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             shear_range=0.1,
                             zoom_range=0.1,
                             horizontal_flip=True,
                             fill_mode='nearest')

Generador de lotes de datos de entrenamiento

In [4]:
train_generator = datagen.flow_from_directory(
    data_directory,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    shuffle=True)

Found 100 images belonging to 3 classes.


Arquitectura pre-entrenada MobileNet sin las capas densas

In [5]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

58889256/58889256 [==============================] - 3s 0us/step


Construccion de la red neuronal

In [6]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(train_generator.num_classes, activation='softmax'))


Congelar las capas base (MobileNet)

In [7]:
for layer in base_model.layers:
    layer.trainable = False

Compilar el modelo

In [14]:
optimizer = Adam(lr=0.001)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Entrenar el modelo

In [ ]:
model.fit(train_generator, epochs=50)

Epoch 1/50
4/4 [==============================] - 67s 20s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 2/50
4/4 [==============================] - 65s 14s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/50
4/4 [==============================] - 63s 14s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/50
4/4 [==============================] - 65s 14s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/50
4/4 [==============================] - 63s 14s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/50
4/4 [==============================] - 64s 14s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/50
4/4 [==============================] - 63s 14s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/50
4/4 [==============================] - 65s 14s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/50
4/4 [==============================] - 63s 14s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/50
4/4 [==============================] - 64s 14s/step - loss: 0.0000e+00 - accura

Funcion para predecir la huella digital

In [10]:
def predict_fingerprint(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (img_width, img_height))
    img = np.expand_dims(img, axis=0) / 255.
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction)
    return predicted_class


Reconocimiento de la huella

In [13]:
image_to_predict = '/content/P5.jpg'
predicted_person = predict_fingerprint(image_to_predict)
print("La huella digital pertenece a la persona:", predicted_person)

1/1 [==============================] - 1s 950ms/step
La huella digital pertenece a la persona: 1
